In [2]:
import fitz  # PyMuPDF
import re
import os

def extrair_informacoes_pdf(pdf_path):
    """
    Extrai o texto da seção 'INFORMAÇÕES DO INTEIRO TEOR' de um PDF.
    Retorna uma lista de blocos encontrados (pois pode haver mais de uma seção no mesmo PDF).
    """

    doc = fitz.open(pdf_path)
    blocos = []
    capturando = False
    buffer_text = ""

    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text("text")

        # Se achamos o título, começamos a capturar
        if "INFORMAÇÕES DO INTEIRO TEOR" in text:
            capturando = True
            start_index = text.index("INFORMAÇÕES DO INTEIRO TEOR") + len("INFORMAÇÕES DO INTEIRO TEOR")
            buffer_text += text[start_index:].strip() + "\n"

        elif capturando:
            # Se encontramos outro cabeçalho que marca o fim, salvamos o bloco e paramos
            if re.search(r"\n(PROCESSO|DESTAQUE|SEGUNDA|TERCEIRA|QUARTA|QUINTA|SEXTA|SÉTIMA|OITAVA)\b", text):
                blocos.append(buffer_text.strip())
                buffer_text = ""
                capturando = False
            else:
                buffer_text += text.strip() + "\n"

    # Se terminou o arquivo mas ainda está capturando, salva o último bloco
    if buffer_text:
        blocos.append(buffer_text.strip())

    # Limpeza básica do texto
    blocos_limpos = []
    for bloco in blocos:
        bloco = re.sub(r"http\S+", "", bloco)  # remove URLs
        bloco = re.sub(r"processo\.stj\.jus\.br.*", "", bloco)  # remove rodapés
        bloco = re.sub(r"\n\d+/\d+\n", " ", bloco)  # remove números de página
        blocos_limpos.append(bloco.strip())

    return blocos_limpos


# Exemplo de uso em um único PDF
pdf_file = "data/Inf0655.pdf"
blocos = extrair_informacoes_pdf(pdf_file)

for i, bloco in enumerate(blocos, start=1):
    print(f"\n=== Bloco {i} extraído de {pdf_file} ===\n")
    print(bloco[:1000])  # Mostra apenas os 1000 primeiros caracteres
    print("\n--- Fim do bloco ---\n")


=== Bloco 1 extraído de data/Inf0655.pdf ===

A  Centrais  Elétricas  Brasileiras  S.A.  -  Eletrobrás  foi  criada  pela  União  em  1961,  na  forma  de
sociedade de economia mista, como holding do setor elétrico, com o objetivo específico previsto no
 art.  2º  da  Lei  n.  3.890-A/1961  de  construir  e  operar  usinas  geradoras/produtoras  e  linhas  de
transmissão e distribuição de energia elétrica. A ideia era superar a crise gerada pela desproporção
entre  a  demanda  e  a  oferta  de  energia  no  país,  ou  seja,  atuar  em  um  setor  estratégico  para  o
desenvolvimento nacional. Nesse contexto, o empréstimo compulsório sobre o consumo de energia
elétrica  instituído  pela  Lei  n.  4.156/1962  foi  uma  forma  de  se  verter  recursos  para  a  Eletrobrás
intervir  no  setor  de  energia  elétrica  subscrevendo  ações,  tomando  obrigações  e  financiando  as
demais empresas atuantes no setor das quais o Poder Público (Federal, Estadual ou Municipal) fosse
acionista. De 

In [3]:
import fitz  # PyMuPDF
import re
import os

def extrair_informacoes_pdf(pdf_path):
    """
    Extrai o(s) bloco(s) de 'INFORMAÇÕES DO INTEIRO TEOR' de um PDF.
    Retorna uma lista com os textos completos.
    """
    doc = fitz.open(pdf_path)
    blocos = []
    capturando = False
    buffer_text = ""

    for page_num in range(len(doc)):
        text = doc[page_num].get_text("text")

        # Se encontramos o título, começamos a capturar
        if "INFORMAÇÕES DO INTEIRO TEOR" in text:
            capturando = True
            start_index = text.index("INFORMAÇÕES DO INTEIRO TEOR") + len("INFORMAÇÕES DO INTEIRO TEOR")
            buffer_text += text[start_index:].strip() + "\n"
            continue

        if capturando:
            # Verifica se nesta página existe um marcador de nova seção
            match = re.search(r"\n(PROCESSO|DESTAQUE|SEGUNDA|TERCEIRA|QUARTA|QUINTA|SEXTA|SÉTIMA|OITAVA)\b", text)
            if match:
                # Finaliza o bloco atual
                blocos.append(buffer_text.strip())
                buffer_text = ""
                capturando = False
            else:
                buffer_text += text.strip() + "\n"

    # Se terminou o PDF ainda capturando, salva o último bloco
    if buffer_text:
        blocos.append(buffer_text.strip())

    # Limpeza do texto (URLs, rodapés, numeração de página)
    blocos_limpos = []
    for bloco in blocos:
        bloco = re.sub(r"http\S+", "", bloco)
        bloco = re.sub(r"processo\.stj\.jus\.br.*", "", bloco)
        bloco = re.sub(r"\n\d+/\d+\n", " ", bloco)  # remove numeração "1/27"
        blocos_limpos.append(bloco.strip())

    return blocos_limpos


# ==== Exemplo de uso ====
pdf_file = "data/Inf0655.pdf"
blocos = extrair_informacoes_pdf(pdf_file)

for i, bloco in enumerate(blocos, 1):
    print(f"\n=== Bloco {i} extraído ===\n")
    print(bloco)
    print("\n--- Fim do bloco ---\n")



=== Bloco 1 extraído ===

A  Centrais  Elétricas  Brasileiras  S.A.  -  Eletrobrás  foi  criada  pela  União  em  1961,  na  forma  de
sociedade de economia mista, como holding do setor elétrico, com o objetivo específico previsto no
 art.  2º  da  Lei  n.  3.890-A/1961  de  construir  e  operar  usinas  geradoras/produtoras  e  linhas  de
transmissão e distribuição de energia elétrica. A ideia era superar a crise gerada pela desproporção
entre  a  demanda  e  a  oferta  de  energia  no  país,  ou  seja,  atuar  em  um  setor  estratégico  para  o
desenvolvimento nacional. Nesse contexto, o empréstimo compulsório sobre o consumo de energia
elétrica  instituído  pela  Lei  n.  4.156/1962  foi  uma  forma  de  se  verter  recursos  para  a  Eletrobrás
intervir  no  setor  de  energia  elétrica  subscrevendo  ações,  tomando  obrigações  e  financiando  as
demais empresas atuantes no setor das quais o Poder Público (Federal, Estadual ou Municipal) fosse
acionista. De relevo que: a) o emp

In [4]:
import fitz  # PyMuPDF
import re
import os

def extrair_informacao_inteiro_teor(pdf_path):
    """
    Extrai apenas o bloco de 'INFORMAÇÕES DO INTEIRO TEOR' que vai
    até a próxima seção (PROCESSO, TURMA, SEÇÃO, INFORMAÇÕES ADICIONAIS, etc).
    Retorna uma lista com 1 bloco por ocorrência.
    """
    doc = fitz.open(pdf_path)
    blocos = []
    capturando = False
    buffer_text = ""

    # regex para detectar o início de nova seção
    regex_nova_secao = re.compile(
        r"\n("
        r"PROCESSO|DESTAQUE|PRIMEIRA|SEGUNDA|TERCEIRA|QUARTA|QUINTA|SEXTA|SÉTIMA|OITAVA"
        r"|SEÇÃO|INFORMAÇÕES ADICIONAIS"
        r")\b"
    )

    for page_num in range(len(doc)):
        text = doc[page_num].get_text("text")

        # Se encontramos o título, começamos a capturar
        if "INFORMAÇÕES DO INTEIRO TEOR" in text and not capturando:
            capturando = True
            start_index = text.index("INFORMAÇÕES DO INTEIRO TEOR") + len("INFORMAÇÕES DO INTEIRO TEOR")
            buffer_text += text[start_index:].strip() + "\n"
            continue

        if capturando:
            # Verifica se encontramos início de nova seção
            match = regex_nova_secao.search(text)
            if match:
                blocos.append(buffer_text.strip())
                buffer_text = ""
                capturando = False
            else:
                buffer_text += text.strip() + "\n"

    # Se terminou o PDF ainda capturando, salva o último bloco
    if buffer_text:
        blocos.append(buffer_text.strip())

    # Limpeza do texto (URLs, rodapés, numeração de página)
    blocos_limpos = []
    for bloco in blocos:
        bloco = re.sub(r"http\S+", "", bloco)
        bloco = re.sub(r"processo\.stj\.jus\.br.*", "", bloco)
        bloco = re.sub(r"\n\d+/\d+\n", " ", bloco)  # remove numeração "1/27"
        blocos_limpos.append(bloco.strip())

    return blocos_limpos


# ==== Exemplo de uso ====
pdf_file = "data/Inf0655.pdf"
blocos = extrair_informacao_inteiro_teor(pdf_file)

for i, bloco in enumerate(blocos, 1):
    print(f"\n=== Bloco {i} extraído ===\n")
    print(bloco)
    print("\n--- Fim do bloco ---\n")



=== Bloco 1 extraído ===

A  Centrais  Elétricas  Brasileiras  S.A.  -  Eletrobrás  foi  criada  pela  União  em  1961,  na  forma  de
sociedade de economia mista, como holding do setor elétrico, com o objetivo específico previsto no
 art.  2º  da  Lei  n.  3.890-A/1961  de  construir  e  operar  usinas  geradoras/produtoras  e  linhas  de
transmissão e distribuição de energia elétrica. A ideia era superar a crise gerada pela desproporção
entre  a  demanda  e  a  oferta  de  energia  no  país,  ou  seja,  atuar  em  um  setor  estratégico  para  o
desenvolvimento nacional. Nesse contexto, o empréstimo compulsório sobre o consumo de energia
elétrica  instituído  pela  Lei  n.  4.156/1962  foi  uma  forma  de  se  verter  recursos  para  a  Eletrobrás
intervir  no  setor  de  energia  elétrica  subscrevendo  ações,  tomando  obrigações  e  financiando  as
demais empresas atuantes no setor das quais o Poder Público (Federal, Estadual ou Municipal) fosse
acionista. De relevo que: a) o emp

In [5]:
import fitz  # PyMuPDF
import re

def extrair_informacoes_pdf(pdf_path):
    """
    Extrai todos os blocos de 'INFORMAÇÕES DO INTEIRO TEOR' de um PDF,
    cada um até a próxima seção (PROCESSO, TURMA, SEÇÃO, INFORMAÇÕES ADICIONAIS, etc).
    Retorna uma lista de blocos.
    """
    doc = fitz.open(pdf_path)
    blocos = []
    capturando = False
    buffer_text = ""

    # regex para início de nova seção
    regex_nova_secao = re.compile(
        r"\n("
        r"PROCESSO|DESTAQUE|PRIMEIRA|SEGUNDA|TERCEIRA|QUARTA|QUINTA|SEXTA|SÉTIMA|OITAVA"
        r"|SEÇÃO|INFORMAÇÕES ADICIONAIS"
        r")\b"
    )

    for page_num in range(len(doc)):
        text = doc[page_num].get_text("text")

        # Se encontramos o título, SEMPRE iniciamos nova captura
        if "INFORMAÇÕES DO INTEIRO TEOR" in text:
            # Se estava capturando e não fechou, salva bloco anterior
            if buffer_text:
                blocos.append(buffer_text.strip())
                buffer_text = ""

            capturando = True
            start_index = text.index("INFORMAÇÕES DO INTEIRO TEOR") + len("INFORMAÇÕES DO INTEIRO TEOR")
            buffer_text += text[start_index:].strip() + "\n"
            continue

        if capturando:
            match = regex_nova_secao.search(text)
            if match:
                blocos.append(buffer_text.strip())
                buffer_text = ""
                capturando = False
            else:
                buffer_text += text.strip() + "\n"

    # se terminou o PDF ainda capturando, salva o último bloco
    if buffer_text:
        blocos.append(buffer_text.strip())

    # limpeza (urls, rodapés, numeração de página)
    blocos_limpos = []
    for bloco in blocos:
        bloco = re.sub(r"http\S+", "", bloco)
        bloco = re.sub(r"processo\.stj\.jus\.br.*", "", bloco)
        bloco = re.sub(r"\n\d+/\d+\n", " ", bloco)  # remove "5/27"
        blocos_limpos.append(bloco.strip())

    return blocos_limpos


# ==== Exemplo de uso ====
pdf_file = "data/Inf0655.pdf"
blocos = extrair_informacoes_pdf(pdf_file)

for i, bloco in enumerate(blocos, 1):
    print(f"\n=== Bloco {i} extraído ===\n")
    print(bloco)
    print("\n--- Fim do bloco ---\n")



=== Bloco 1 extraído ===

A  Centrais  Elétricas  Brasileiras  S.A.  -  Eletrobrás  foi  criada  pela  União  em  1961,  na  forma  de
sociedade de economia mista, como holding do setor elétrico, com o objetivo específico previsto no
 art.  2º  da  Lei  n.  3.890-A/1961  de  construir  e  operar  usinas  geradoras/produtoras  e  linhas  de
transmissão e distribuição de energia elétrica. A ideia era superar a crise gerada pela desproporção
entre  a  demanda  e  a  oferta  de  energia  no  país,  ou  seja,  atuar  em  um  setor  estratégico  para  o
desenvolvimento nacional. Nesse contexto, o empréstimo compulsório sobre o consumo de energia
elétrica  instituído  pela  Lei  n.  4.156/1962  foi  uma  forma  de  se  verter  recursos  para  a  Eletrobrás
intervir  no  setor  de  energia  elétrica  subscrevendo  ações,  tomando  obrigações  e  financiando  as
demais empresas atuantes no setor das quais o Poder Público (Federal, Estadual ou Municipal) fosse
acionista. De relevo que: a) o emp

In [31]:
import pdfplumber
import re
import os
import pandas as pd

def extracao_infos2(pasta, arquivo_resultado):

    df_lido = pd.read_excel(arquivo_resultado)
    arquivos_lidos = df_lido['ARQUIVO'].unique()
    arquivos_totais = os.listdir(pasta)

    resultados = []
    for file in arquivos_totais:

        if file in arquivos_lidos:
            continue
        else:
            
            print(f"Lendo arquivo: {file}")
            pdf_path = os.path.join(pasta, file)

            # regex para PROCESSO, RAMO, TEMA, DESTAQUE
            pattern_infos = re.compile(
                r"PROCESSO\s+(.*?)\s+RAMO DO DIREITO\s+(.*?)\s+TEMA\s+(.*?)\s+DESTAQUE\s+(.*?)(?=INFORMAÇÕES DO INTEIRO TEOR)",
                re.DOTALL
            )

            # regex para INFORMAÇÕES DO INTEIRO TEOR (até próxima seção)
            pattern_inteiro_teor = re.compile(
                r"INFORMAÇÕES DO INTEIRO TEOR\s+(.*?)(?=(?:\n(?:PROCESSO|DESTAQUE|PRIMEIRA|SEGUNDA|TERCEIRA|QUARTA|QUINTA|SEXTA|SÉTIMA|OITAVA|SEÇÃO|INFORMAÇÕES ADICIONAIS)|\Z))",
                re.DOTALL
            )

            # extrair texto de todas as páginas
            with pdfplumber.open(pdf_path) as pdf:
                all_text = ""
                for page in pdf.pages:
                    all_text += page.extract_text() + "\n"

            # aplicar regex PROCESSO, RAMO, TEMA, DESTAQUE
            matches_infos = pattern_infos.findall(all_text)

            # aplicar regex INTEIRO TEOR
            matches_inteiro_teor = pattern_inteiro_teor.findall(all_text)

            # pode haver mais de um bloco de inteiro teor no mesmo PDF
            for i, (processo, ramo, tema, destaque) in enumerate(matches_infos):
                inteiro_teor = matches_inteiro_teor[i] if i < len(matches_inteiro_teor) else ""

                dados = {
                    "PROCESSO": " ".join(processo.split()),
                    "RAMO_DO_DIREITO": " ".join(ramo.split()),
                    "TEMA": " ".join(tema.split()),
                    "DESTAQUE": " ".join(destaque.split()),
                    "INTEIRO_TEOR": " ".join(inteiro_teor.split()),
                    "ARQUIVO": file
                }
                resultados.append(dados)
                print("Bloco extraído com sucesso!")

    df_novo = pd.DataFrame(resultados)

    if df_novo.empty:
        return 'Todos os arquivos já haviam sido lidos.'
    else:
        df_novo['LINK'] = 'https://processo.stj.jus.br/SCON/GetPDFINFJ?edicao=' + df_novo['ARQUIVO'].str.replace('Inf','').str.replace('.pdf','')
        df_final = pd.concat([df_lido, df_novo])
        df_final.to_excel("output/jurisprudencia_extraida.xlsx", index=False)
        return 'Arquivo jurisprudencia_extraida.xlsx atualizado com as informações dos novos PDFs.'


In [36]:
extracao_infos2('data','output/jurisprudencia_extraida.xlsx')

Lendo arquivo: Inf0025E.pdf
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Lendo arquivo: Inf0639.pdf
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com sucesso!
Bloco extraído com su

'Arquivo jurisprudencia_extraida.xlsx atualizado com as informações dos novos PDFs.'

In [ ]:
import os
import pdfplumber
import pandas as pd

arquivos_totais = ['']

resultados = []

pdf_path = os.path.join('data', 'Inf0025E.pdf')

# regex para PROCESSO, RAMO, TEMA, DESTAQUE
pattern_infos = re.compile(
    r"PROCESSO\s+(.*?)\s+RAMO DO DIREITO\s+(.*?)\s+TEMA\s+(.*?)\s+DESTAQUE\s+(.*?)(?=INFORMAÇÕES DO INTEIRO TEOR)",
    re.DOTALL
)

# regex para INFORMAÇÕES DO INTEIRO TEOR (até próxima seção)
pattern_inteiro_teor = re.compile(
    r"INFORMAÇÕES DO INTEIRO TEOR\s+(.*?)(?=(?:\n(?:PROCESSO|DESTAQUE|PRIMEIRA|SEGUNDA|TERCEIRA|QUARTA|QUINTA|SEXTA|SÉTIMA|OITAVA|SEÇÃO|INFORMAÇÕES ADICIONAIS)|\Z))",
    re.DOTALL
)

# extrair texto de todas as páginas
with pdfplumber.open(pdf_path) as pdf:
    all_text = ""
    for page in pdf.pages:
        all_text += page.extract_text() + "\n"

# aplicar regex PROCESSO, RAMO, TEMA, DESTAQUE
matches_infos = pattern_infos.findall(all_text)

# aplicar regex INTEIRO TEOR
matches_inteiro_teor = pattern_inteiro_teor.findall(all_text)

# pode haver mais de um bloco de inteiro teor no mesmo PDF
for i, (processo, ramo, tema, destaque) in enumerate(matches_infos):
    inteiro_teor = matches_inteiro_teor[i] if i < len(matches_inteiro_teor) else ""

    dados = {
        "PROCESSO": " ".join(processo.split()),
        "RAMO_DO_DIREITO": " ".join(ramo.split()),
        "TEMA": " ".join(tema.split()),
        "DESTAQUE": " ".join(destaque.split()),
        "INTEIRO_TEOR": " ".join(inteiro_teor.split()),
        #"ARQUIVO": file
    }
    resultados.append(dados)
    print("Bloco extraído com sucesso!")


In [26]:
x = pd.DataFrame(resultados)

In [30]:
x['INTEIRO_TEOR']

0     A suspensão de liminar, medida excepcional de ...
1     O alcance da Lei n. 12.846/2013, denominada de...
2     A cassação da aposentadoria não viola o princí...
3     Trata-se de controvérsia que diz respeito à co...
4     No caso, a parte prestou concurso para Profess...
5     O art. 4º, § 2º, da Lei n. 12.846/2013 (Lei An...
6     A controvérsia envolve o debate acerca da legi...
7     Consiste a controvérsia em definir se a expres...
8     Cinge-se a controvérsia em definir se o Etanol...
9     A controvérsia tem origem em ação declaratória...
10    A controvérsia cinge-se em saber o percentual ...
11    A base de cálculo da CPRB está prevista no art...
12    A discussão apresentada refere-se ao suposto d...
13    Cinge-se a controvérsia a saber se, em concurs...
14    processo.stj.jus.br/jurisprudencia/externo/inf...
15    A controvérsia reside em definir se é admissív...
16    Na origem, a controvérsia diz respeito à legal...
17    processo.stj.jus.br/jurisprudencia/externo

In [29]:
display(x['INTEIRO_TEOR'][17])

'processo.stj.jus.br/jurisprudencia/externo/informativo/ 27/32 Informativo de Jurisprudência n. 25 - Edição Extraordinária 15 de julho de 2025 O art. 38 do Código Tributário Nacional (CTN) dispõe expressamente que a base de cálculo do ITCD é o valor venal dos bens ou direitos transmitidos. Nesse diapasão, a jurisprudência do Superior Tribunal de Justiça entende que a base de cálculo do ITCMD é o valor venal dos bens e direitos transmitidos, assim compreendido como aquele que corresponde ao valor de mercado. O fisco poderá afastar o montante declarado pelo contribuinte, apurado com base no valor patrimonial, obtido com a divisão do patrimônio líquido da sociedade pela quantidade de quotas representativas do capital integralizado, quando verificar que não foram apurados isoladamente os valores de mercado dos bens imóveis que integralizaram esse capital, na forma do art. 148 do CTN. Apurar a exação tendo como base unicamente o valor patrimonial das quotas sociais atribuídas pelos sócios, 